# GraphVAE for networks generation

### Variational Autoencoders

First of all let's have a brief remind on **VAE** Variational autoencoders, firstly introduced by [*D. P. Kingma and M. Welling. 'Auto-encoding variational bayes', 2014*](https://arxiv.org/pdf/1312.6114.pdf)

VAE is a neural network architecture belonging to the family of variational Bayesian methods.

From a probabilistic point of view we want to maximize the likelyhood of our data **x** given a proper set of parameters **$\theta$**, like in a normal MLE problem: $p_{\theta}(x) = p(x|\theta)$. By neglecting from the third moment upwards, we could approximate the distribution to a normal distribution $\mathcal{N}(x|\mu,\sigma)$. Simple distributions like the normal ones are usually easy to maximize, however if we assume a prior over a latent space $z$ the posterior usually becomes intractable.

By marginalizing over $z$ we obtain:

$$p_{\theta}(x) = \int_{\mathcal{Z}}{p_{\theta}(x,z)dz} = \int_{\mathcal{Z}}{p_{\theta}(x|z)p_{\theta}(z)dz}$$

So we may define the set of relationships between the input data and the latent space through:
- $p_{\theta}(z)$ the prior distribution of the latent space
- $p_{\theta}(x|z)$ the likelyhood
- $p_{\theta}(z|x)$ the posterior

Using the Bayes's theorem we could get:

$$p_{\theta}(z|x) = \frac{p_{\theta}(x|z)p_{\theta}(z)}{p_{\theta}(x)}$$

but the the computation is usually expensive if not intractable. However, it is possible to approximate the posterior:

$$ q_{\phi}(z|x)\simeq p_{\theta}(z|x)$$

### Variational Graph Autoencoders

Variational Graph Autoencoders [Kingma and Welling, 2016](https://arxiv.org/pdf/1611.07308.pdf) provide a framework extension to graph for VAEs.

Our problem could be formalized as follows: an undirected graph $\mathcal{G}=(\nu, \epsilon)$ with $N$ nodes and a features/attribute matrix $X\in\mathbb{R}^{N\times C}$. An adjacency matrix $A\in\mathbb{R}^{N\times N}$ with self-loops included. Assume that each node within the graph is associated to a latent variable $\in Z$ with $Z\in\mathbb{R}^{N\times F}$ and $F$ being the latent space dimension, we are interested in inferring the latent variables of nodes in the graph and decoding the edges.

Similarly to VAE, VGAE consist of an **encoder** $q_{\phi}(Z|A,X)$, a **decoder** $p_{\theta}(A|Z)$ and a prior $p(Z)$.
- The **encoder** tries to learn a distribution of latent variables associated with each node conditioning on the node features $X$ and $A$. One efficient option is to instantiate $q_{\phi}(Z|A,X)$ as a graph neural network where the learnable parameters are $\phi$. In particular, VGAE assumes a node-independent encoder so that the probabilities factorize: $$q_{\phi}(Z|A,X) = \prod_{i=1}^{N}q_{\phi}(z_{i}|A,X)$$ then, by neglecting from the third moment upwards of your distribution, the problem translates into: $$q_{\phi}(z_{i}|A,X)=\mathcal{N}(z_{i}|\mu_{i},diag(\sigma_{i}^2))$$ $$\mathbf{\mu},\mathbf{\sigma} = GCN_{\phi}(X,A)$$ Where $z_{i}, \mu_{i},\sigma_{i}$ are the i-th rows of the matrices $Z,\mu$ and $\sigma$. The mean and diagonal covariance are predicted by the encoder network, i.e. the $GCN$. For a two-layer $GCN$ we have: $$ H=\tilde{A}\sigma{(\tilde{A}XW_{1})}W_{2}$$ where $H\in\mathbb{R}^{N\times d_{H}}$ are the node representations (each node is associated with a size $d_{H}$ vector), $\tilde{A}=D^{-\frac{1}{2}}(A+I)D^{-\frac{1}{2}}$ is the normalized adjacency matrix as described by the [original 2016 GCN paper by Kipf and Welling](https://arxiv.org/abs/1609.02907). $\sigma$ is a pointwise nonlinearity (e.g. a ReLU) and $\{W_{1},W_{2}\}$ are trainable parameters containing the biases. Relying on the learned node representation, the distribution is computed as follows: $$q_{\phi}(Z|A,X) = \prod_{i=1}^{N}q_{\phi}(z_{i}|A,X)$$ $$q_{\phi}(z_{i}|A,X)=\mathcal{N}(z_{i}|\mu_{i},\sigma_{i}^2I)$$ $$\mu=MPL_{\mu}(H)$$ $$\log{\sigma}=MPL_{\sigma}{(H)}$$ Where $\mu_{i},\sigma_{i}$ are the i-th rows of the MPL predictions. Therefore, the set $\phi$ of parameters consist in the set of the trainable parameters of the twp MLPs and the aforementioned GCN. We remark that the NNs underlying each Gaussian ('GNN+MLP') are very powerful so that the conditional distributions are expressive in capturing the uncertainty of latent variables and computationally cheaper than other techniques.
- GVAEs often adopt a **prior** that remains fixed during the training. A common choice is a node-independent Gaussian as follows: $$p(Z)=\prod_{i}^{N}{p(z_{i})}$$ $$p(z_i)=\mathcal{N}(0,I)$$ Surely this prior can be substituted by more powerful models such as autoregressive models at the cost of more computational resources. Nevertheless, a simple prior like the one expressed before is usually the starting point to benchmark more complicated alternatives.
- The aim of a **decoder** is to construct a probability distribution over the graph and it's features/attributes conditioned on the latent variables, $p(\mathcal{G}|Z)$. One should always consider all the possible node permutations, each corresponding to an adjacency matrix with different rows orderings which leaves the graph unchanged: $$ p(\mathcal{G}|Z) = \sum_{P\in\prod_{\mathcal{G}}} {p(PAP^{T},PX|Z)}$$ but we'll neglect this discussion for the moment. A simple and popular construction of the probability distribution could be: $$ p(A,X|Z)=\prod_{i,j}p(A_{ij}|Z)\prod_{i=1}^{N}p(x_i|Z)$$ $$p(A_{ij}|Z)=Bernoulli(\Theta_{ij})$$ $$p(x_i|Z)=\mathcal{N}(\tilde{\mu}_{i},\tilde{\sigma}_i)$$ Where, once again, the parameters are learned through MLPs: $$\Theta_{ij}=MLP_{\Theta}([z_{i}||z_j])$$ $$\tilde{\mu}_{i}=MLP_{\tilde{\mu}}(z_i)$$ $$\tilde{\sigma}_{i}=MLP_{\tilde{\sigma}}(z_i)$$
- The **objective** of the GVAE is the evidence lower bound (ELBO): $$\max_{\theta,\phi}{\mathbb{E}_{q_{\phi}(Z|A,X)} {[\log{p_{\theta}(\mathcal{G}|Z)}} - KL(q_{\phi}(Z|A,X)||p(Z))]}$$ where the Kullback-Leibler divergence measures the divergence between two probability distributions

## Implementing a VGAE for link prediction

For **link prediction** the decoder is simply the dot product of the sampled latent space. This is because our aim is to approximate and adjacency matrix $A$ and the idea is that if nodes $u,v$ are similar, their representations $z_{u}, z_{v}$ should be similar as well:
- similar nodes: $z_{u}^{T}z_{v}$ should be maximal
- different nodes: $z_{u}^{T}z_{v}$ should be minimal
So far we assumed that similar nodes should be connected, thats why matrix factorization $$A\simeq\hat{A} = Z^{T}Z$$ works as an approximation of A.

As we want the elements of $\hat{A}$ to be as similar as possible to the ones of $A$, part of the objective is then a measure of how well the model reconstructs the original matrix and is computed through a **binary cross entropy loss**. We will call it *reconstruction term* $\mathcal{L}_{rec}$: $$ \mathcal{L_rec} = \sum_{i,j \in V} {-A_{ij}\log{\hat{A}_{ij}} - (1-A_{ij})\log{(1-\hat{A_{ij}})}}$$

The second term that forms the objective of this task is a regularization term used in variational autoencoders to promote the latent space to follow a specific distribution, usually a normal distribution $z\sim\mathcal{N}(0,\mathbb{I})$. As a matter of facts, the KL-divergence measures the divergence between the learned distribution of the latent variables and target distribution and we will call this term $\mathcal{L}_{KL}$: 

$$\mathcal{L}_{KL}=KL(q_{\phi}(Z|A)||p(Z)) = \frac{1}{2}\sum_{i}{(1+\log{\sigma_{i}^{2}}-\mu_{i}^{2}-\sigma_{i}^{2})}$$

Where $\mu, \sigma$ are the encoder's output. The total loss, called *evidence lower bound (ELBO)* loss is therefore: $$\mathcal{L}_{ELBO} = \mathcal{L}_{rec} - \mathcal{L}_{KL}$$

In [3]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

#Set libraries seed
np.random.seed(0)
torch.manual_seed(0)

#Set GPU device
device = torch.device('mps')

### Dataset

We will make use of the `Cora` dataset which is contained in the `Planetoid` data laoder of `PyTorch`.
It is one of the most pupular datasets used for node classification and link prediction and represents a network of 2708 publications, where each connection is a reference.
Each publication is described as a binary vector of 1433 words (the the features matrix is $X\in\mathbb{R}^{2708\times1433}$), where 0 and 1 indicate the absence or presence of the corrisponding word, respectively (aka *binary bag* of words). In terms of node classification, vertices can be classified in 7 different categories.

In [4]:
dataset = Planetoid('.', name = 'Cora')
data = dataset[0]
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

Where `x` is the feature matrix, `edge_index` is a `(2, num_edges)` tensor (Compressed Sparse Column format) where edges are stored column-wise. We could print other interesting quantities:

In [5]:
print(f'Dataset: {dataset}')
print('-'*15)
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of node features: {data.x.shape[1]}')  # o anche data.num_node_features
print(f'Number of edge features: {data.num_edge_features}')
print(f'Number of edges: {data.num_edges}')
print(f'Number of classes: {dataset.num_classes}')

Dataset: Cora()
---------------
Number of graphs: 1
Number of nodes: 2708
Number of node features: 1433
Number of edge features: 0
Number of edges: 10556
Number of classes: 7


`train_mask`, `val_mask` and `test_mask` are `num_nodes` long masks applied to the dataset for the train/val/test split.
The default split in the Cora dataset (as used in Planetoid) follows a transductive setting, which means that during training, you only have access to the training nodes and their edges and the adjacency matrix will be restricted to reflect the connections among the nodes in the training set.
This approach mimics a semi-supervised learning setting, where you aim to generalize your model's performance on unseen nodes (validation and test sets) based on the information available in the training set.

The nodes in the training set for the Cora dataset (and other datasets in PyTorch Geometric's Planetoid) are typically chosen consecutively based on the order in which they appear in the dataset. In other words, the training set consists of a contiguous block of nodes taken from the beginning of the dataset.

This consecutive selection of nodes ensures that the training set forms a representative subset of the dataset, and the ordering of nodes in the dataset is preserved. It is a common practice for simplicity and consistency when working with such datasets.

Now, instead of using the default split in Cora, we implement a `transform` object that normalizes input feautures and directly performs tensor device conversion and randomly splits links in a 85/5/10 division.

In [6]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val = 0.05, num_test=0.1, is_undirected=True, split_labels=True, add_negative_train_samples=False),
])
 #add_negative_train_samples = False -> model already performs negative sampling

As we are approaching a link prediction task, it is essential for us to split out data using `T.RandomLinkSplit`  such that the training split does not include edges in validation and test splits; and the validation split does not include edges in the test split.

Now we can load Cora dataset with the transformation:

In [7]:
dataset = Planetoid('.', name = 'Cora', transform=transform)
train_data, val_data, test_data = dataset[0]
train_data

Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[4488], pos_edge_label_index=[2, 4488])

- `positive_edges` are associated in the context of link prediction to known, existing edges in the graph.
- `negative edges` (not present in the training set) are edges that do not exist in the graph amd are typically sampled during training to create a balanced dataset for training the model.

### Model

Let's import the models:

In [8]:
from torch_geometric.nn import GCNConv, VGAE

First of all we implement the encoder which should be composed of three GCN layers: 1 shared input layer, a second layer to approximate mean values $\mu_{i}$ and a third one for the variance (actually $\log{\sigma}$).
**Please note** that the architecture is quite flexible and other type of GNNs convolutions (GraphSAGE, GIN) could be therefore used.

In [9]:
class Encoder(torch.nn.Module):

    def __init__(self, dim_in, dim_out):
        super().__init__()                                  #overwrite the method of the parent class
        self.conv1 = GCNConv(dim_in, 2 * dim_out)           #dim_in: dimension of feature space, dim_out: dimension of latent space
        self.conv_mu = GCNConv(2 * dim_out, dim_out)
        self.conv_logstd = GCNConv(2 * dim_out, dim_out)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

We initialize the VGAE layer with the Encoder as input. By default, the Decoder is set to be the inner product, which is actually what we need to perform link prediction. In this particular case, the VGAE pytorch implementation **does not include MLPs after the GCNs**.

In [10]:
model = VGAE(Encoder(dataset.num_features, 16)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
model

VGAE(
  (encoder): Encoder(
    (conv1): GCNConv(1433, 32)
    (conv_mu): GCNConv(32, 16)
    (conv_logstd): GCNConv(32, 16)
  )
  (decoder): InnerProductDecoder()
)

The `.train()` function firstly computes the embedding matrix Z through `model.encode()` which despite of the name, simply samples embeddings from the learned distribution. Secondly, the ELBO loss is computed with `model.recon_loss()` (binary crossentropy loss, what we called $\mathcal{L}_{rec}$) and model.kl_loss(), the $\mathcal{L}_{KL}$ loss. The decoder is implicitly called by the model to calculate the cross-entropy loss.

In [13]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    loss = model.recon_loss(z, train_data.pos_edge_label_index) + (1/train_data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)

**Negative sampling**:

*positive edges* are edges that actually exist in my graph while *negative edges* are artificially created edges that are not present in the original graph. During training the model is presented the positive edges and it is instructed to learn to predict their existence by adjusting its parameters to minimize the prediction error.

During evaluation, to assess the model's performance, we create negative edges by sampling pairs of node that were not connected in the original graph and we mix them with the positive known edges to form a test set for evaluation. The model task during evaluation is to distinguish between true edges and fake ones.

To this purpose, we use metrics like area under the ROC curve (**AUC**) and average precision (**AP**) by putting a 0.5 threshold to our newly generated approximation of the adjacency matrix $\hat{A}$ and counting the correct prediction over the negative and positive edges set.

Then we could define a `test()` function which simply calls the VGAE's dedicated method:

In [11]:
@torch.no_grad()       #explicitly states that this function should be executed without gradient tracking
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    return model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)

In [14]:
for epoch in range(1000):
    loss = train()
    val_auc, val_ap = test(val_data)
    if epoch % 50 == 0:
        print(f'Epoch: {epoch:>3} | Loss: {loss:.4f} | Val AUC: {val_auc:.4f} | Val AP: {val_ap:.4f}')

Epoch:   0 | Loss: 3.4849 | Val AUC: 0.6796 | Val AP: 0.7303
Epoch:  50 | Loss: 1.3280 | Val AUC: 0.6666 | Val AP: 0.7179
Epoch: 100 | Loss: 1.2012 | Val AUC: 0.7703 | Val AP: 0.7827
Epoch: 150 | Loss: 1.1269 | Val AUC: 0.7861 | Val AP: 0.7945
Epoch: 200 | Loss: 1.0276 | Val AUC: 0.8350 | Val AP: 0.8313
Epoch: 250 | Loss: 0.9941 | Val AUC: 0.8578 | Val AP: 0.8556
Epoch: 300 | Loss: 0.9425 | Val AUC: 0.8904 | Val AP: 0.8891
Epoch: 350 | Loss: 0.9346 | Val AUC: 0.8961 | Val AP: 0.8992
Epoch: 400 | Loss: 0.9130 | Val AUC: 0.9004 | Val AP: 0.9066
Epoch: 450 | Loss: 0.9070 | Val AUC: 0.8943 | Val AP: 0.9059
Epoch: 500 | Loss: 0.9018 | Val AUC: 0.9038 | Val AP: 0.9157
Epoch: 550 | Loss: 0.8947 | Val AUC: 0.9028 | Val AP: 0.9144
Epoch: 600 | Loss: 0.8986 | Val AUC: 0.9015 | Val AP: 0.9136
Epoch: 650 | Loss: 0.8860 | Val AUC: 0.8965 | Val AP: 0.9109
Epoch: 700 | Loss: 0.8897 | Val AUC: 0.9011 | Val AP: 0.9135
Epoch: 750 | Loss: 0.8843 | Val AUC: 0.9033 | Val AP: 0.9164
Epoch: 800 | Loss: 0.886

In [15]:
test_auc, test_ap = test(test_data)
print(f'Test AUC: {test_auc:.4f} | Test AP: {test_ap:.4f}')

Test AUC: 0.9124 | Test AP: 0.9160


Finally we have out approximated adjacency matrix $\hat{A}$ for the `test_data` portion of nodes:

In [16]:
z = model.encode (test_data.x, test_data.edge_index)
Ahat = torch.sigmoid(z @ z.T)
Ahat

tensor([[0.9463, 0.5546, 0.6930,  ..., 0.6464, 0.7617, 0.7717],
        [0.5546, 0.9301, 0.9180,  ..., 0.2188, 0.5944, 0.5480],
        [0.6930, 0.9180, 0.9482,  ..., 0.1936, 0.8068, 0.7456],
        ...,
        [0.6464, 0.2188, 0.1936,  ..., 0.8876, 0.3795, 0.4563],
        [0.7617, 0.5944, 0.8068,  ..., 0.3795, 0.8707, 0.8264],
        [0.7717, 0.5480, 0.7456,  ..., 0.4563, 0.8264, 0.7933]],
       device='mps:0', grad_fn=<SigmoidBackward0>)

Training a VGAE is fast and outputs are easily understandable, however we know that GCNs are not the most expressive layers.

## Implementing a GraphVAE for graph generation

In a **graph generation** task we need to specialize our **encoder** to a feed-forward neural network with edge-conditional graph convolutions (ECC) and the **decoder** to a MLP with three outputs.

The problem consists in having a graph $G=(A,E,F)$ with A adj matrix, E edge attribute tensor and F node attribute matrix. Once again, we will assume a normal prior $p(z)=\mathcal{N}(0,\mathbb{I})$ and the whole model (encoder $\rightarrow$ prior sampling $\rightarrow$ decoder) is trained using: 
$$ \mathcal{L}_{\theta,\phi;G} = \mathbb{E}_{q_{\phi}(z|G)} [-\log{p_{\theta}(G|z)}] + KL[q_{\phi}(z|G)||p(z)] $$

The decoder itself is a deterministic MLP with three output in the last layer. Sigmoid activation function is used to compute $\hat{A}$, whereas edge- and node-wise softmax is applied to obtain $\hat{E}$ and $\hat{F}$, respectively. The actual formulation of the loss should consider permutation caveats that we wont discuss here. The following implementation was proposed by [*J. You et al.*](https://github.com/JiaxuanYou/graph-generation/tree/master) as a baseline model for comparison with their newly introduced GraphRNN model.

In [35]:
import torch

In [36]:
class GraphVAE(torch.nn.Module):

    def __init__(self, input_dim, hidden_dim, latent_dim, max_num_nodes, pool='sum'):
        
        '''
        Args:
            input_dim: input feature dimension for node.
            hidden_dim: hidden dim for 2-layer gcn.
            latent_dim: dimension of the latent representation of graph.
        '''

        super().__init()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.bn1 = torch.nn.BatchNorm1d(input_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.bn2 = torch.nn.BatchNorm1d(input_dim)
        self.act = torch.nn.Relu()

SyntaxError: expected ':' (2521976885.py, line 1)